## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [24]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [25]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [26]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [27]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210120.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [28]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,244620,6800,251420,174279,0.693179,4568,2021-01-19
1,Aragón,41265,1200,42465,33734,0.794395,623,2021-01-19
2,Asturias,44195,1200,45395,36054,0.794228,1687,2021-01-19
3,Baleares,22590,600,23190,15697,0.676887,167,2021-01-19
4,Canarias,46185,1300,47485,43599,0.918164,339,2021-01-20
5,Cantabria,20625,500,21125,19473,0.921799,0,2021-01-20
6,Castilla y Leon,93315,2500,95815,74350,0.775975,633,2021-01-19
7,Castilla La Mancha,61875,1700,63575,48755,0.766890,195,2021-01-19
8,Cataluña,211220,5800,217020,162830,0.750300,913,2021-01-19
9,C. Valenciana,109000,2900,111900,99932,0.893047,1737,2021-01-20


..hacemos una copia para hacerle las transformaciones necesarias:

In [29]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [30]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [31]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [32]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [33]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [34]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [35]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [36]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [37]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,20/1/21,Andalucía,244620,6800,251420,174279,0.693179,4568,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,20/1/21,Aragón,41265,1200,42465,33734,0.794395,623,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,20/1/21,Asturias,44195,1200,45395,36054,0.794228,1687,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,20/1/21,Baleares,22590,600,23190,15697,0.676887,167,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,20/1/21,Canarias,46185,1300,47485,43599,0.918164,339,20/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,20/1/21,Cantabria,20625,500,21125,19473,0.921799,0,20/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,20/1/21,Castilla y Leon,93315,2500,95815,74350,0.775975,633,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,20/1/21,Castilla La Mancha,61875,1700,63575,48755,0.766890,195,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,20/1/21,Cataluña,211220,5800,217020,162830,0.750300,913,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,20/1/21,C. Valenciana,109000,2900,111900,99932,0.893047,1737,20/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [38]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [39]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [40]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
215,19/1/21,Navarra,17700.0,500.0,18200,12991,0.713791,299.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
216,19/1/21,País Vasco,47205.0,1400.0,48605,28069,0.577492,390.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
217,19/1/21,Ceuta,1005.0,200.0,1205,1080,0.896266,29.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
218,19/1/21,Melilla,1005.0,200.0,1205,1297,1.076349,29.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [41]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [42]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [43]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [44]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
235,20/1/21,Navarra,20625.0,500.0,21125,14559,0.689183,503.0,NaN,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
236,20/1/21,País Vasco,55980.0,1400.0,57380,33523,0.584228,390.0,NaN,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
237,20/1/21,Ceuta,1980.0,200.0,2180,1110,0.509174,29.0,NaN,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
238,20/1/21,Melilla,1980.0,200.0,2180,1326,0.608257,29.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [45]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [46]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)